# Neptune + TensorBoard

There are two ways of working with Neptune and TensorBoard:

* convert your TensorBoard runs from the logs directory with the bash command: 

```bash
neptune tensorboard /path/to/logs
```

* log TensorBoard runs live by adding a snippet a snippet at the top of your script:

```python
neptune_tensorboard.integrate_with_tensorflow()
```

# Convert TensorBoard logs to Neptune experiments

## Before you start

### Install Dependencies

In [ ]:
! pip install --quiet tensorboard==2.4.0 tensorflow==2.3.1 neptune-tensorboard==0.5.1 neptune-client==0.4.131

In [ ]:
! pip install --quiet --upgrade tensorboard neptune-tensorboard neptune-client

### Create some TensorBoard logs

**Note:**  
You can run it once to create a single TensorBoard log or multiple times.

In [ ]:
import tensorflow as tf
import datetime

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
      ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

## Step 1: Set your Environment Variables

In [ ]:
%env NEPTUNE_API_TOKEN=ANONYMOUS
%env NEPTUNE_PROJECT=shared/tensorboard-integration

**Note:**   
Instead of logging data to the public project 'shared/tensorboard-integration' as an anonymous user 'neptuner' you can log it to your own project.

To do that:

1. Get your Neptune API token

![image](https://neptune.ai/wp-content/uploads/get_token.gif)

2. Pass the token to `NEPTUNE_API_TOKEN` environment variable
3. Create a new Neptune Project (learn how to do that [here](https://docs.neptune.ai/workspace-project-and-user-management/projects/create-project.html))
4. Pass your username and project_name to the `NEPTUNE_PROJECT` environment variable

For example:

```
%env NEPTUNE_API_TOKEN=eyJhcGlfYW908fsdf23f940jiri0bn3085gh03riv03irn
%env NEPTUNE_PROJECT=jakub-czakon/tensorboard_logs
```

## Step 2: Convert TensorBoard logs to Neptune experiments

Run the following command

In [ ]:
! neptune tensorboard logs

**Note:**  
You can specify the path to the directory where your TensorBoard run logs are.

```bash
neptune tensorboard /path/to/logs
```

## See converted experiments

Click on the link(s) above to browse the TensorBoard run in Neptune or go to [shared/tensorflow-integration project](https://ui.neptune.ai/o/shared/org/tensorboard-integration/experiments?viewId=def2c858-3510-4bf9-9e52-8720fadecb11).

**Note:**  
Running the above code pushes all your TensorBoard logs to Neptune. You can rerun the code after running fresh experiments, and only they will be pushed.

# Log runs live to Neptune via TensorBoard

## Step 1: Initialize Neptune

In [ ]:
import neptune

neptune.init(api_token='ANONYMOUS', project_qualified_name='shared/tensorboard-integration')

Neptune gives you an option of logging data under a public folder as an anonymous user. This is great when you are just trying out the application and don't have a Neptune account yet.  

---

If you already have a [Neptune account](https://neptune.ai/register), you can create your own experiment and start logging to it using your personal API token. Pass your `username` to the `project_qualified_name` argument of the `neptune.init()` method: `project_qualified_name='YOUR_USERNAME/YOUR_PROJECT_NAME`. If you don't have a project yet, keep `/sandbox` at the end. The `sandbox` project is automatically created for you.

## Step 2: Create an experiment

In [ ]:
neptune.create_experiment('tensorboard-logging')

## Step 3: Run ``neptune_tensorboard.integrate_with_tensorflow()``

In [ ]:
import neptune_tensorboard

neptune_tensorboard.integrate_with_tensorflow()

## Step 4: Add your training code

In [ ]:
import tensorflow as tf
import datetime

mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
      ])

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

## Step 5: Explore results in the Neptune UI

Go to the experiment link and explore charts that were created from the metric values you logged. 

In [ ]:
# tests

exp = neptune.get_experiment()

In [ ]:
neptune.stop()

In [ ]:
# tests

all_logs = exp.get_logs()

## check logs
correct_logs = ['loss', 'accuracy', 'val_loss', 'val_accuracy']

if set(all_logs.keys()) != set(correct_logs):
    raise ValueError('incorrect metrics')

# More logging options

## Create an experiment and train a model

In [ ]:
neptune.create_experiment('tensorboard-more-logging-options')
neptune_tensorboard.integrate_with_tensorflow()

model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=5, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

## Log model weights

In [ ]:
model.save('my_model')

neptune.log_artifact('my_model')

## Log interactive charts

### Install neptune-contrib

In [ ]:
! pip install --quiet neptune-contrib==0.24.9 matplotlib==3.2.0 scikit-plot==0.3.7 plotly==4.12.0

In [ ]:
! pip install --quiet neptune-contrib scikit-plot --upgrade

### Create chart

In [ ]:
import matplotlib.pyplot as plt 
from scikitplot.metrics import plot_roc

y_test_pred = model.predict(x_test)

fig, ax = plt.subplots()
plot_roc(y_test, y_test_pred, ax=ax)

### Log chart to Neptune as interactive Plotly chart

In [ ]:
from neptunecontrib.api import log_chart

log_chart(name='ROC curve', chart=fig)

In [ ]:
# tests

exp = neptune.get_experiment()

In [ ]:
neptune.stop()

In [ ]:
# tests

all_logs = exp.get_logs()

## check logs
correct_logs = ['loss', 'accuracy', 'val_loss', 'val_accuracy']

if set(all_logs.keys()) != set(correct_logs):
    raise ValueError('incorrect metrics')